In [ ]:
from pathlib import Path
import json, cv2, numpy as np
from joblib import load

# === paths (edit if needed) ===
DATA_ROOT = Path(r"C:\Users\ACER NITRO 5 GAMING\.cache\kagglehub\datasets\nexuswho\laboro-tomato\versions\5")
ANN = json.loads((DATA_ROOT/"annotations"/"test.json").read_text())   # or "train.json"

# === load model ===
model = load("svm_lab.joblib")
le    = load("label_encoder.joblib")

def extract_lab_features(bgr_img, mask_bool):
    lab = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2LAB)
    if mask_bool is None: mask_bool = np.ones(lab.shape[:2], dtype=bool)
    if mask_bool.sum() < 10: return None
    L = lab[:,:,0][mask_bool].astype(np.float32)
    a = lab[:,:,1][mask_bool].astype(np.float32)
    b = lab[:,:,2][mask_bool].astype(np.float32)
    return np.array([L.mean(), L.std()+1e-6, a.mean(), a.std()+1e-6, b.mean(), b.std()+1e-6], dtype=np.float32)

def bbox_to_mask(h, w, bbox):
    x,y,bw,bh = bbox
    x0,y0,x1,y1 = int(x), int(y), int(x+bw), int(y+bh)
    m = np.zeros((h,w), dtype=bool)
    m[y0:y1, x0:x1] = True
    return m

# pick one image & its first annotation
img_info = ANN["images"][0]
# your dump: test.json images are under val/images; train.json under train/images
img_path = DATA_ROOT / ("val/images" if (DATA_ROOT/"val/images").exists() else "test/images") / img_info["file_name"]
im = cv2.imread(str(img_path))

anns = [a for a in ANN["annotations"] if a["image_id"] == img_info["id"]]
ann = anns[0]
mask = bbox_to_mask(im.shape[0], im.shape[1], ann["bbox"])
feats = extract_lab_features(im, mask).reshape(1,-1)

pred_idx = model.predict(feats)[0]
pred_label = le.inverse_transform([pred_idx])[0]
print("Predicted ripeness:", pred_label)

# visualize
x,y,w,h = map(int, ann["bbox"])
cv2.rectangle(im, (x,y), (x+w, y+h), (0,255,0), 2)
cv2.putText(im, pred_label, (x, max(10,y-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2, cv2.LINE_AA)
cv2.imshow("Prediction", im); cv2.waitKey(0)


Predicted ripeness: half_ripened
